In [1]:
import warnings
warnings.filterwarnings('ignore')

# **Imports**

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import MinMaxScaler
import scipy


In [ ]:
!pip install prophet
!pip install xgboost

# **Descarga de los datasets**

In [3]:
train = pd.read_csv('./UH_2023_TRAIN.txt', sep = "|")
ETO_dataset = pd.read_csv('./DATOS_ETO.TXT', sep = "|")
METEO_dataset = pd.read_csv('./DATOS_METEO.TXT', sep = "|")

## DATASET TRAIN

In [4]:
train_original = train.copy()


In [5]:
train["MODO"] = train["MODO"]-1 #Vemos que modo varia entre 2 y 1, le restamos para pasarlo a binario

#### ALTITUD

In [6]:
train["ALTITUD"] = train["ALTITUD"].str[:3].apply(pd.to_numeric)
altitud_actualizada = train.sort_values(["ID_ESTACION","ID_ZONA"],
                                        na_position='last')['ALTITUD'].fillna(method='ffill').fillna(method="bfill")
train["ALTITUD"] = altitud_actualizada
del altitud_actualizada

#### SUPERFICIE 20/21/22

In [7]:
corregir_superficie_20 = train[train["CAMPAÑA"] == 20]
corregir_superficie_20 = pd.get_dummies(corregir_superficie_20, columns = ["ID_ESTACION","VARIEDAD"])
corregir_superficie_20["SUPERFICIE"].replace(0, np.nan, inplace=True)
corregir_superficie_20.drop(["CAMPAÑA","ID_FINCA","COLOR","TIPO","ID_ZONA"],axis=1,inplace=True)

In [8]:
scaler = MinMaxScaler(feature_range = (0, 2))
scaler.fit(corregir_superficie_20["PRODUCCION"].to_numpy().reshape(-1, 1))
corregir_superficie_20["PRODUCCION"] = scaler.transform(corregir_superficie_20["PRODUCCION"].to_numpy().reshape(-1, 1))

scaler2 = MinMaxScaler(feature_range=(0,1))
scaler2.fit(corregir_superficie_20["ALTITUD"].to_numpy().reshape(-1,1))
corregir_superficie_20["ALTITUD"]= scaler2.transform(corregir_superficie_20["ALTITUD"].to_numpy().reshape(-1,1))

imputer = KNNImputer(n_neighbors = 3)
corregir_superficie_20["SUPERFICIE"] = imputer.fit_transform(corregir_superficie_20)[:,2]

corregir_superficie_20["PRODUCCION"] = scaler.inverse_transform(corregir_superficie_20["PRODUCCION"].to_numpy().reshape(-1,1))
corregir_superficie_20["ALTITUD"] = scaler2.inverse_transform(corregir_superficie_20["ALTITUD"].to_numpy().reshape(-1,1))

train.loc[train.CAMPAÑA == 20, 'SUPERFICIE'] = corregir_superficie_20["SUPERFICIE"]


In [9]:
corregir_superficie_21 = train[train["CAMPAÑA"] == 21]
corregir_superficie_21 = corregir_superficie_21.drop(["CAMPAÑA","ID_FINCA","ID_ZONA","TIPO","COLOR"], axis = 1)
corregir_superficie_21 = pd.get_dummies(corregir_superficie_21, columns=["ID_ESTACION","VARIEDAD"])
corregir_superficie_21["SUPERFICIE"].replace(0, np.nan, inplace=True)

scaler.fit(corregir_superficie_21["PRODUCCION"].to_numpy().reshape(-1,1))
corregir_superficie_21["PRODUCCION"] = scaler.transform(corregir_superficie_21["PRODUCCION"].to_numpy().reshape(-1,1))

scaler2.fit(corregir_superficie_21["ALTITUD"].to_numpy().reshape(-1,1))
corregir_superficie_21["ALTITUD"] = scaler2.transform(corregir_superficie_21["ALTITUD"].to_numpy().reshape(-1,1))

corregir_superficie_21["SUPERFICIE"] = imputer.fit_transform(corregir_superficie_21)[:,2]

corregir_superficie_21["PRODUCCION"] = scaler.inverse_transform(corregir_superficie_21["PRODUCCION"].to_numpy().reshape(-1,1))
corregir_superficie_21["ALTITUD"] = scaler2.inverse_transform(corregir_superficie_21["ALTITUD"].to_numpy().reshape(-1,1))

train.loc[train.CAMPAÑA == 21, 'SUPERFICIE'] = corregir_superficie_21["SUPERFICIE"]

In [10]:
corregir_superficie_21["ID_FINCA"] = train[train["CAMPAÑA"] == 21]["ID_FINCA"]
corregir_superficie_21["VARIEDAD"] = train[train["CAMPAÑA"] == 21]["VARIEDAD"]


corregir_superficie_20["ID_FINCA"] = train[train["CAMPAÑA"] == 20]["ID_FINCA"]
corregir_superficie_20["VARIEDAD"] = train[train["CAMPAÑA"] == 20]["VARIEDAD"]
keys = list(corregir_superficie_20[["ID_FINCA","VARIEDAD","MODO"]].columns.values)
i1 = corregir_superficie_20.set_index(keys).index
i2 = corregir_superficie_21.set_index(keys).index


igualtats_20 = corregir_superficie_20[["ID_FINCA","VARIEDAD","MODO","SUPERFICIE"]][i1.isin(i2)]
igualtats_21 = corregir_superficie_21[["ID_FINCA","VARIEDAD","MODO","SUPERFICIE"]][i2.isin(i1)]


In [11]:
igualtats_20["SUPERFICIE_21"] = train[train["CAMPAÑA"]==21]["SUPERFICIE"][i2.isin(i1)].values

igualtats_20["SUPERFICIE"] = igualtats_20[["SUPERFICIE","SUPERFICIE_21"]].mean(axis=1)
igualtats_21["SUPERFICIE"] = igualtats_20["SUPERFICIE"].values
train.loc[igualtats_20.index, "SUPERFICIE"] = igualtats_20["SUPERFICIE"]
train.loc[igualtats_21.index, "SUPERFICIE"] = igualtats_21["SUPERFICIE"]

In [12]:
corregir_superficie_22 = train[train["CAMPAÑA"] == 22]
corregir_superficie_22 = corregir_superficie_22[corregir_superficie_22["SUPERFICIE"] == 0]
keys = list(corregir_superficie_22[["ID_FINCA","VARIEDAD","MODO"]].columns.values)
i1 = corregir_superficie_22.set_index(keys).index
i2 = igualtats_20.set_index(keys).index
corregir_superficie_22["SUPERFICIE"] = (igualtats_20[i2.isin(i1)]["SUPERFICIE"]).values
train.loc[corregir_superficie_22.index, 'SUPERFICIE'] = corregir_superficie_22["SUPERFICIE"]

#### OTRAS SUPERFICIES

In [13]:
campaña_21 = train[train["CAMPAÑA"] == 21] 
campaña_20 = train[train["CAMPAÑA"] == 20] 
campaña_15 = train[train["CAMPAÑA"] == 15] 
campaña_16 = train[train["CAMPAÑA"] == 16] 
campaña_17 = train[train["CAMPAÑA"] == 17] 
campaña_18 = train[train["CAMPAÑA"] == 18] 
campaña_19 = train[train["CAMPAÑA"] == 19] 
campaña_14 = train[train["CAMPAÑA"] == 14] 

keys = list(train[["ID_FINCA","VARIEDAD","MODO","ID_ESTACION","TIPO"]].columns.values)
i_14 = campaña_14.set_index(keys).index
i_20 = campaña_20.set_index(keys).index
i_21 = campaña_21.set_index(keys).index
i_15 = campaña_15.set_index(keys).index
i_16 = campaña_16.set_index(keys).index
i_17 = campaña_17.set_index(keys).index
i_18 = campaña_18.set_index(keys).index
i_19 = campaña_19.set_index(keys).index

In [14]:
sub_train = train[train["CAMPAÑA"] < 22].copy()
terrenos = sub_train[["ID_FINCA","VARIEDAD","MODO","ID_ESTACION","TIPO"]].drop_duplicates()
terrenos = terrenos[(terrenos.ID_ESTACION != 1) & (terrenos.ID_ESTACION != 13) & (terrenos.ID_ESTACION != 0)
                   & (terrenos.ID_ESTACION != 17) & (terrenos.ID_ESTACION != 11) & (terrenos.ID_ESTACION != 4)
                    & (terrenos.ID_ESTACION != 2) & (terrenos.ID_ESTACION != 6) & (terrenos.ID_ESTACION != 8)]
i_terrenos = terrenos.set_index(keys).index


In [15]:
#14   
terrenos = terrenos.merge(campaña_14[i_14.isin(i_terrenos)][['ID_FINCA','VARIEDAD','MODO','ID_ESTACION','PRODUCCION','TIPO']], how='left',on = ['ID_FINCA','VARIEDAD','MODO','ID_ESTACION','TIPO'])
terrenos = terrenos.rename(columns={"PRODUCCION":"PRODUCCION_14"})
#15
terrenos = terrenos.merge(campaña_15[i_15.isin(i_terrenos)][['ID_FINCA','VARIEDAD','MODO','ID_ESTACION','PRODUCCION','TIPO']], how='left',on = ['ID_FINCA','VARIEDAD','MODO','ID_ESTACION','TIPO'])
terrenos = terrenos.rename(columns={"PRODUCCION":"PRODUCCION_15"})
#16
terrenos = terrenos.merge(campaña_16[i_16.isin(i_terrenos)][['ID_FINCA','VARIEDAD','MODO','ID_ESTACION','PRODUCCION','TIPO']], how='left',on = ['ID_FINCA','VARIEDAD','MODO','ID_ESTACION','TIPO'])
terrenos = terrenos.rename(columns={"PRODUCCION":"PRODUCCION_16"})
#17
terrenos = terrenos.merge( campaña_17[i_17.isin(i_terrenos)][['ID_FINCA','VARIEDAD','MODO','ID_ESTACION','PRODUCCION','TIPO']], how='left',on = ['ID_FINCA','VARIEDAD','MODO','ID_ESTACION','TIPO'])
terrenos = terrenos.rename(columns={"PRODUCCION":"PRODUCCION_17"})
#18
terrenos = terrenos.merge( campaña_18[i_18.isin(i_terrenos)][['ID_FINCA','VARIEDAD','MODO','ID_ESTACION','PRODUCCION','TIPO']], how='left',on = ['ID_FINCA','VARIEDAD','MODO','ID_ESTACION','TIPO'])
terrenos = terrenos.rename(columns={"PRODUCCION":"PRODUCCION_18"})
#19
terrenos = terrenos.merge(campaña_19[i_19.isin(i_terrenos)][['ID_FINCA','VARIEDAD','MODO','ID_ESTACION','PRODUCCION','TIPO']], how='left',on = ['ID_FINCA','VARIEDAD','MODO','ID_ESTACION','TIPO'])
terrenos = terrenos.rename(columns={"PRODUCCION":"PRODUCCION_19"})
#20
terrenos = terrenos.merge( campaña_20[i_20.isin(i_terrenos)][['ID_FINCA','VARIEDAD','MODO','ID_ESTACION','PRODUCCION','SUPERFICIE','TIPO']], how='left',on = ['ID_FINCA','VARIEDAD','MODO','ID_ESTACION','TIPO'])
terrenos = terrenos.rename(columns={"PRODUCCION":"PRODUCCION_20", "SUPERFICIE":"SUPERFICIE_20"})
#21
terrenos = terrenos.merge( campaña_21[i_21.isin(i_terrenos)][['ID_FINCA','VARIEDAD','MODO','ID_ESTACION','PRODUCCION','SUPERFICIE','TIPO']], how='left',on = ['ID_FINCA','VARIEDAD','MODO','ID_ESTACION','TIPO'])
terrenos = terrenos.rename(columns={"PRODUCCION":"PRODUCCION_21","SUPERFICIE":"SUPERFICIE_21"})
terrenos = terrenos.fillna(0)

In [16]:
aux = terrenos.copy()
aux["DIF_14_20"] = 0
aux["DIF_15_20"] = 0
aux["DIF_16_20"] = 0
aux["DIF_17_20"] = 0
aux["DIF_18_20"] = 0
aux["DIF_19_20"] = 0
aux["DIF_14_21"] = 0
aux["DIF_15_21"] = 0
aux["DIF_16_21"] = 0
aux["DIF_17_21"] = 0
aux["DIF_18_21"] = 0
aux["DIF_19_21"] = 0

Test estadístico de igualdad de varianza

In [17]:
from numpy import inf

for i in np.unique(terrenos.ID_ESTACION.values):
    prod_20 = terrenos[terrenos["ID_ESTACION"] == i]["PRODUCCION_20"]
    prod_21 = terrenos[terrenos["ID_ESTACION"] == i]["PRODUCCION_21"]

    #CAMPAÑA 14
    prod_14 = terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_14"]
    if(scipy.stats.levene(prod_14[prod_14!=0],prod_20[prod_20!=0]).pvalue>0.2):
        diferencias_20 = terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_14"]/ terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_20"]
        diferencias_20[diferencias_20 == inf]=0
        normalizado_20 = (diferencias_20[diferencias_20!=0] - np.mean(diferencias_20[diferencias_20!=0]))/np.std(diferencias_20[diferencias_20!=0])
        aux.loc[normalizado_20.index,"DIF_14_20"] = np.abs(normalizado_20.fillna(0))
        
    if(scipy.stats.levene(prod_14[prod_14!=0],prod_21[prod_21!=0]).pvalue>0.2):
        diferencias_21 = terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_14"]/ terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_21"]
        diferencias_21[diferencias_21 == inf]=0
        normalizado_21 = (diferencias_21[diferencias_21!=0] - np.mean(diferencias_21[diferencias_21!=0]))/np.std(diferencias_21[diferencias_21!=0])
        aux.loc[normalizado_21.index,"DIF_14_21"] = np.abs(normalizado_21.fillna(0))
    
    #CAMPAÑA 15
    prod_15 = terrenos[terrenos["ID_ESTACION"] == i]["PRODUCCION_15"]
    if(scipy.stats.levene(prod_15[prod_15 != 0],prod_20[prod_20 != 0]).pvalue > 0.2):
        diferencias_20 = terrenos[terrenos["ID_ESTACION"] == i]["PRODUCCION_15"] / terrenos[terrenos["ID_ESTACION"] == i]["PRODUCCION_20"]
        diferencias_20[diferencias_20 == inf] = 0
        normalizado_20 = (diferencias_20[diferencias_20 != 0] - np.mean(diferencias_20[diferencias_20 != 0])) / np.std(diferencias_20[diferencias_20 != 0])
        aux.loc[normalizado_20.index,"DIF_15_20"] = np.abs(normalizado_20.fillna(0))
    
    
    if(scipy.stats.levene(prod_15[prod_15!=0],prod_21[prod_21!=0]).pvalue > 0.2):
        diferencias_21 = terrenos[terrenos["ID_ESTACION"] == i]["PRODUCCION_15"] / terrenos[terrenos["ID_ESTACION"] == i]["PRODUCCION_21"]
        diferencias_21[diferencias_21 == inf] = 0
        normalizado_21 = (diferencias_21[diferencias_21 != 0] - np.mean(diferencias_21[diferencias_21 != 0]))/np.std(diferencias_21[diferencias_21 != 0])
        aux.loc[normalizado_21.index,"DIF_15_21"] = np.abs(normalizado_21.fillna(0))
    
    #CAMPAÑA 16
    prod_16 = terrenos[terrenos["ID_ESTACION"] == i]["PRODUCCION_16"]
    if(scipy.stats.levene(prod_16[prod_16!=0],prod_20[prod_20 != 0]).pvalue > 0.2):
        diferencias_20 = terrenos[terrenos["ID_ESTACION"] == i]["PRODUCCION_16"]/ terrenos[terrenos["ID_ESTACION"] == i]["PRODUCCION_20"]
        diferencias_20[diferencias_20 == inf]=0
        normalizado_20 = (diferencias_20[diferencias_20!=0] - np.mean(diferencias_20[diferencias_20!=0]))/np.std(diferencias_20[diferencias_20!=0])
        aux.loc[normalizado_20.index,"DIF_16_20"] = np.abs(normalizado_20.fillna(0))
    
    if(scipy.stats.levene(prod_16[prod_16!=0],prod_21[prod_21!=0]).pvalue>0.2):
        diferencias_21 = terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_16"]/ terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_21"]
        diferencias_21[diferencias_21 == inf]=0
        normalizado_21 = (diferencias_21[diferencias_21!=0] - np.mean(diferencias_21[diferencias_21!=0]))/np.std(diferencias_21[diferencias_21!=0])
        aux.loc[normalizado_21.index,"DIF_16_21"] = np.abs(normalizado_21.fillna(0))
   
    #CAMPAÑA 17
    prod_17 = terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_17"]
    if(scipy.stats.levene(prod_17[prod_17!=0],prod_20[prod_20!=0]).pvalue>0.2):
        diferencias_20 = terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_17"]/ terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_20"]
        diferencias_20[diferencias_20 == inf]=0
        normalizado_20 = (diferencias_20[diferencias_20!=0] - np.mean(diferencias_20[diferencias_20!=0]))/np.std(diferencias_20[diferencias_20!=0])
        aux.loc[normalizado_20.index,"DIF_17_20"] = np.abs(normalizado_20.fillna(0))
    
    
    if(scipy.stats.levene(prod_17[prod_17!=0],prod_21[prod_21!=0]).pvalue>0.2):
        diferencias_21 = terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_17"]/ terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_21"]
        diferencias_21[diferencias_21 == inf]=0
        normalizado_21 = (diferencias_21[diferencias_21!=0] - np.mean(diferencias_21[diferencias_21!=0]))/np.std(diferencias_21[diferencias_21!=0])
        aux.loc[normalizado_21.index,"DIF_17_21"] = np.abs(normalizado_21.fillna(0))
    
    #CAMPAÑA 18
    prod_18 = terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_18"]
    if(scipy.stats.levene(prod_18[prod_18!=0],prod_20[prod_20!=0]).pvalue>0.2):
        diferencias_20 = terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_18"]/ terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_20"]
        diferencias_20[diferencias_20 == inf]=0
        normalizado_20 = (diferencias_20[diferencias_20!=0] - np.mean(diferencias_20[diferencias_20!=0]))/np.std(diferencias_20[diferencias_20!=0])
        aux.loc[normalizado_20.index,"DIF_18_20"] = np.abs(normalizado_20.fillna(0))
    
    
    if(scipy.stats.levene(prod_18[prod_18!=0],prod_21[prod_21!=0]).pvalue>0.2):
        diferencias_21 = terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_18"]/ terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_21"]
        diferencias_21[diferencias_21 == inf]=0
        normalizado_21 = (diferencias_21[diferencias_21!=0] - np.mean(diferencias_21[diferencias_21!=0]))/np.std(diferencias_21[diferencias_21!=0])
        aux.loc[normalizado_21.index,"DIF_18_21"] = np.abs(normalizado_21.fillna(0))
    
    #CAMPAÑA 19
    prod_19 = terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_19"]
    if(scipy.stats.levene(prod_19[prod_19!=0],prod_20[prod_20!=0]).pvalue>0.2):
        diferencias_20 = terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_19"]/ terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_20"]
        diferencias_20[diferencias_20 == inf]=0
        normalizado_20 = (diferencias_20[diferencias_20!=0] - np.mean(diferencias_20[diferencias_20!=0]))/np.std(diferencias_20[diferencias_20!=0])
        aux.loc[normalizado_20.index,"DIF_19_20"] = np.abs(normalizado_20.fillna(0))
    
    
    if(scipy.stats.levene(prod_19[prod_19!=0],prod_21[prod_21!=0]).pvalue>0.2):
        diferencias_21 = terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_19"]/ terrenos[terrenos["ID_ESTACION"]==i]["PRODUCCION_21"]
        diferencias_21[diferencias_21 == inf]=0
        normalizado_21 = (diferencias_21[diferencias_21!=0] - np.mean(diferencias_21[diferencias_21!=0]))/np.std(diferencias_21[diferencias_21!=0])
        aux.loc[normalizado_21.index,"DIF_19_21"] = np.abs(normalizado_21.fillna(0))
    

In [18]:
aux = aux.drop(["PRODUCCION_14","PRODUCCION_15","PRODUCCION_16","PRODUCCION_17","PRODUCCION_18","PRODUCCION_19","PRODUCCION_20","PRODUCCION_21"],axis=1)
#Evitamos reemplazar los 0 de MODO y TIPO por NaN
aux_modos = aux["MODO"] 
aux_tipo = aux["TIPO"]
aux.replace(0, np.nan, inplace=True)
aux["MODO"] = aux_modos
aux["TIPO"]= aux_tipo
train_bo = train.copy()
train_bo.SUPERFICIE.replace(0,np.nan,inplace=True)


68-95-99.7, optamos por el 68, es decir el 1

In [19]:
for i in aux.values:
    
    if i[7] < 1:
        train_bo.loc[(train_bo.CAMPAÑA == 14) & (train_bo.ID_FINCA == i[0]) & (train_bo.VARIEDAD == i[1])
                    & (train_bo.MODO == i[2]) & (train_bo.ID_ESTACION == i[3]) 
                     & (train_bo.TIPO == i[4]), 'SUPERFICIE'] = i[5]
    elif i[13]< 1:
        train_bo.loc[(train_bo.CAMPAÑA == 14) & (train_bo.ID_FINCA == i[0]) & (train_bo.VARIEDAD == i[1])
                    & (train_bo.MODO == i[2]) & (train_bo.ID_ESTACION == i[3]) 
                     & (train_bo.TIPO == i[4]), 'SUPERFICIE'] = i[6]
    if i[8] < 1:
        train_bo.loc[(train_bo.CAMPAÑA == 15) & (train_bo.ID_FINCA == i[0]) & (train_bo.VARIEDAD == i[1])
                    & (train_bo.MODO == i[2]) & (train_bo.ID_ESTACION == i[3]) 
                     & (train_bo.TIPO == i[4]), 'SUPERFICIE'] = i[5]
    elif i[14]< 1:
        train_bo.loc[(train_bo.CAMPAÑA == 15) & (train_bo.ID_FINCA == i[0]) & (train_bo.VARIEDAD == i[1])
                    & (train_bo.MODO == i[2]) & (train_bo.ID_ESTACION == i[3]) 
                     & (train_bo.TIPO == i[4]), 'SUPERFICIE'] = i[6]
    if i[9] < 1:
        train_bo.loc[(train_bo.CAMPAÑA == 16) & (train_bo.ID_FINCA == i[0]) & (train_bo.VARIEDAD == i[1])
                    & (train_bo.MODO == i[2]) & (train_bo.ID_ESTACION == i[3]) 
                     & (train_bo.TIPO == i[4]), 'SUPERFICIE'] = i[5]
    elif i[15]< 1:
        train_bo.loc[(train_bo.CAMPAÑA == 16) & (train_bo.ID_FINCA == i[0]) & (train_bo.VARIEDAD == i[1])
                    & (train_bo.MODO == i[2]) & (train_bo.ID_ESTACION == i[3]) 
                     & (train_bo.TIPO == i[4]), 'SUPERFICIE'] = i[6]
    if i[10] < 1:
        train_bo.loc[(train_bo.CAMPAÑA == 17) & (train_bo.ID_FINCA == i[0]) & (train_bo.VARIEDAD == i[1])
                    & (train_bo.MODO == i[2]) & (train_bo.ID_ESTACION == i[3]) 
                     & (train_bo.TIPO == i[4]), 'SUPERFICIE'] = i[5]
    elif i[16]< 1:
        train_bo.loc[(train_bo.CAMPAÑA == 17) & (train_bo.ID_FINCA == i[0]) & (train_bo.VARIEDAD == i[1])
                    & (train_bo.MODO == i[2]) & (train_bo.ID_ESTACION == i[3]) 
                     & (train_bo.TIPO == i[4]), 'SUPERFICIE'] = i[6]
    if i[11] < 1:
        train_bo.loc[(train_bo.CAMPAÑA == 18) & (train_bo.ID_FINCA == i[0]) & (train_bo.VARIEDAD == i[1])
                    & (train_bo.MODO == i[2]) & (train_bo.ID_ESTACION == i[3]) 
                     & (train_bo.TIPO == i[4]), 'SUPERFICIE'] = i[5]
    elif i[17]< 1:
        train_bo.loc[(train_bo.CAMPAÑA == 18) & (train_bo.ID_FINCA == i[0]) & (train_bo.VARIEDAD == i[1])
                    & (train_bo.MODO == i[2]) & (train_bo.ID_ESTACION == i[3]) 
                     & (train_bo.TIPO == i[4]), 'SUPERFICIE'] = i[6]
    if i[12] < 1:
        train_bo.loc[(train_bo.CAMPAÑA == 19) & (train_bo.ID_FINCA == i[0]) & (train_bo.VARIEDAD == i[1])
                    & (train_bo.MODO == i[2]) & (train_bo.ID_ESTACION == i[3]) 
                     & (train_bo.TIPO == i[4]), 'SUPERFICIE'] = i[5]
    elif i[18]< 1:
        train_bo.loc[(train_bo.CAMPAÑA == 19) & (train_bo.ID_FINCA == i[0]) & (train_bo.VARIEDAD == i[1])
                    & (train_bo.MODO == i[2]) & (train_bo.ID_ESTACION == i[3]) 
                     & (train_bo.TIPO == i[4]), 'SUPERFICIE'] = i[6]

Llenamos si la producción es muy parecida

In [20]:
superficie_null = train_bo[train_bo["SUPERFICIE"].isnull()]
superficie_nonull = train_bo[(train_bo["SUPERFICIE"]>0) & (train_bo["CAMPAÑA"]<22)]
i_null = superficie_null.set_index(keys).index
i_nonull = superficie_nonull.set_index(keys).index

superficie_null=(superficie_null[i_null.isin(i_nonull)]).drop(["ID_ZONA","ALTITUD","COLOR"],axis=1)
superficie_nonull=superficie_nonull[i_nonull.isin(i_null)].drop(["CAMPAÑA","ID_ZONA","ALTITUD","COLOR"],axis=1)

for i in superficie_null.values:    
    prod_sup = superficie_nonull.loc[(superficie_nonull.ID_FINCA == i[1]) & (superficie_nonull.VARIEDAD == i[3])
                    & (superficie_nonull.MODO == i[4]) & (superficie_nonull.ID_ESTACION == i[2]) 
                     & (superficie_nonull.TIPO == i[5]), ["PRODUCCION","SUPERFICIE"]]
    minimo = np.abs(prod_sup["PRODUCCION"] - i[-1]).min() #Resultado mas parecido entre producción de superfície desconocida y conocida
    index = np.argmin(np.abs(prod_sup["PRODUCCION"] - i[-1]))
    #Si la producción no ha variado en un 50% o la superfície es igual para todas las fincas y hay más de 3 casos, llenamos con esa superf´cie
    if(minimo/prod_sup["PRODUCCION"].values[index] < 0.6) or (((prod_sup["SUPERFICIE"]).min() == prod_sup["SUPERFICIE"].max()) and (len(prod_sup["SUPERFICIE"])>3)):
        train_bo.loc[(train_bo.CAMPAÑA == i[0]) &  (train_bo.ID_FINCA == i[1]) & (train_bo.VARIEDAD == i[3])
                    & (train_bo.MODO == i[4]) & (train_bo.ID_ESTACION == i[2]) 
                     & (train_bo.TIPO == i[5]), 'SUPERFICIE'] = prod_sup["SUPERFICIE"].values[index]

Llenamos con superficies del 22

In [21]:
superficie_null = train_bo[train_bo["SUPERFICIE"].isnull()]
superficie_nonull = train_bo[(train_bo["SUPERFICIE"]>0) & (train_bo["CAMPAÑA"]<22)]
superficie_nonull_22 = train_bo[(train_bo["SUPERFICIE"]>0) & (train_bo["CAMPAÑA"]==22)]
i_null = superficie_null.set_index(keys).index
i_nonull = superficie_nonull.set_index(keys).index
i_nonull_22 = superficie_nonull_22.set_index(keys).index
superficie_null = superficie_null[i_null.isin(i_nonull_22)]
llista = []
for x,index in zip(superficie_null.values,superficie_null.index):
    valor =  superficie_nonull_22.loc[(superficie_nonull_22.ID_FINCA == x[1]) & (superficie_nonull_22.VARIEDAD == x[5])
                       & (superficie_nonull_22.MODO == x[6]) & (superficie_nonull_22.TIPO == x[7]),"SUPERFICIE"]
    superficie_null.loc[(superficie_null.CAMPAÑA==x[0]) & (superficie_null.ID_FINCA == x[1]) & (superficie_null.VARIEDAD == x[5])
                       & (superficie_null.MODO == x[6]) & (superficie_null.TIPO == x[7]),"SUPERFICIE"] = valor.values[0]

In [22]:
train_bo.loc[superficie_null.index,"SUPERFICIE"] = superficie_null["SUPERFICIE"]


Llenamos con un KNN imput

In [23]:
scaler = MinMaxScaler(feature_range=(0,2))
scaler2 = MinMaxScaler(feature_range=(0,1))
imputer = KNNImputer(n_neighbors=5)

def corregir_superficie_2(data,año):
    data = data[data["CAMPAÑA"] == año]
    data = pd.get_dummies(data, columns = ["ID_ESTACION","VARIEDAD"])
    data["SUPERFICIE"].replace(0, np.nan, inplace=True)
    data.drop(["CAMPAÑA","ID_FINCA","COLOR","TIPO","ID_ZONA"],axis=1,inplace=True)
    scaler.fit(data["PRODUCCION"].to_numpy().reshape(-1, 1))
    data["PRODUCCION"] = scaler.transform(data["PRODUCCION"].to_numpy().reshape(-1, 1))
    scaler2.fit(data["ALTITUD"].to_numpy().reshape(-1,1))
    data["ALTITUD"]= scaler2.transform(data["ALTITUD"].to_numpy().reshape(-1,1))
    imputer = KNNImputer(n_neighbors = 3)
    data["SUPERFICIE"] = imputer.fit_transform(data)[:,2]
    train_bo.loc[train_bo.CAMPAÑA == año, 'SUPERFICIE'] = data["SUPERFICIE"]

train_bo_copia = train_bo.copy()
#Evitamos imputar con los nuevos valores añadidos
corregir_superficie_2(train_bo_copia.copy(),14)
corregir_superficie_2(train_bo_copia.copy(),15)
corregir_superficie_2(train_bo_copia.copy(),16)
corregir_superficie_2(train_bo_copia.copy(),17)
corregir_superficie_2(train_bo_copia.copy(),18)
corregir_superficie_2(train_bo_copia.copy(),19)

#### Outliers

La producción se puede explicar mayoritariamente por la superfície, y lo interesante es encontrar los demás factores que nos pueden ayudar a explicar la producción.
Dado que los datos que podemos añadir son climatológicos, que en este caso estan representados de manera indirecta por las estaciones y la campaña, nos es de interés saber la variación de la producción por hectárea por campaña y estacion

Observamos unos cuantos outliers en lo que a producción por hectárea se refiere

In [24]:
from scipy import stats

train_varianza = train_bo.copy()
train_varianza["PRODUCCION"] = train_varianza["PRODUCCION"]/train_bo["SUPERFICIE"]
train_varianza = train_varianza[train_varianza["CAMPAÑA"]<22]
from scipy import stats
z = np.abs(stats.zscore(train_varianza["PRODUCCION"]))
#indices = z[(z>1.9)].index

indices = np.where(z>1.9)[0]
outliers = train_bo.iloc[indices]
train_varianza = train_varianza.drop(index=indices)
train_bo = train_bo.drop(index=indices)


In [25]:
np.where(z>1.9)[0]

array([ 383,  574,  615, 1321, 1718, 1772, 1976, 2201, 2241, 2648, 2825,
       3141, 3317, 3326, 3873, 3925, 4345, 4947, 4993, 5406, 6063, 7090])

In [26]:
indices = train_varianza.index[train_varianza["PRODUCCION"]<200]
train_bo = train_bo.drop(index=indices)


### Feature Engineering Estimación Producción por hectárea

#### Estimacion Campaña 22 con 20 y 21

In [27]:
train_bo["ESTIMACION"] = 0

In [28]:
fincas_22 = train_bo[train_bo["CAMPAÑA"]==22]
fincas_21 = train_bo[train_bo["CAMPAÑA"]==21]
fincas_20 = train_bo[train_bo["CAMPAÑA"]==20]

keys = list(train_bo[["ID_FINCA","VARIEDAD","MODO","TIPO","SUPERFICIE"]].columns.values)

i_22 = fincas_22.set_index(keys).index
i_21 = fincas_21.set_index(keys).index
i_20 = fincas_20.set_index(keys).index

fincas_iguales = i_22[i_22.isin(i_21[i_21.isin(i_20)])]
lista_indices = []

for fincas in fincas_iguales:
    finca_20 = fincas_20.loc[(fincas_20.ID_FINCA == fincas[0]) & (fincas_20.VARIEDAD == fincas[1]) & (fincas_20.MODO == fincas[2])
                            &(fincas_20.TIPO==fincas[3]) & (fincas_20.SUPERFICIE == fincas[4])]
    finca_21 = fincas_21.loc[(fincas_21.ID_FINCA == fincas[0]) & (fincas_21.VARIEDAD == fincas[1]) & (fincas_21.MODO == fincas[2])
                            &(fincas_21.TIPO==fincas[3]) & (fincas_21.SUPERFICIE == fincas[4])]
    finca_22 = fincas_22.loc[(fincas_22.ID_FINCA == fincas[0]) & (fincas_22.VARIEDAD == fincas[1]) & (fincas_22.MODO == fincas[2])
                            &(fincas_22.TIPO==fincas[3]) & (fincas_22.SUPERFICIE == fincas[4])]
    if((np.abs(finca_20["PRODUCCION"].values-finca_21["PRODUCCION"].values) <5000) | (np.abs(finca_20["PRODUCCION"].values/fincas[4]-finca_21["PRODUCCION"].values/fincas[4])<3500)):
        estimacion = (finca_20["PRODUCCION"].values/fincas[4]+finca_21["PRODUCCION"].values/fincas[4])/2
        train_bo.loc[finca_20.index,"ESTIMACION"] = estimacion
        train_bo.loc[finca_21.index,"ESTIMACION"] = estimacion
        train_bo.loc[finca_22.index,"ESTIMACION"] = estimacion
        fincas_20.loc[finca_20.index,"ESTIMACION"] = estimacion
        fincas_21.loc[finca_21.index,"ESTIMACION"] = estimacion
        fincas_22.loc[finca_22.index,"ESTIMACION"] = estimacion


In [29]:
fincas_22= fincas_22.loc[fincas_22.ESTIMACION == 0]
keys = list(train_bo[["ID_FINCA","VARIEDAD","MODO","TIPO"]].columns.values)

i_22 = fincas_22.set_index(keys).index
i_21 = fincas_21.set_index(keys).index
i_20 = fincas_20.set_index(keys).index

fincas_iguales_1 = i_22[i_22.isin(i_21[i_21.isin(i_20)])]
lista_indices = []

for fincas in fincas_iguales_1:
    finca_20 = fincas_20.loc[(fincas_20.ID_FINCA == fincas[0]) & (fincas_20.VARIEDAD == fincas[1]) & (fincas_20.MODO == fincas[2])
                            &(fincas_20.TIPO==fincas[3])]
    finca_21 = fincas_21.loc[(fincas_21.ID_FINCA == fincas[0]) & (fincas_21.VARIEDAD == fincas[1]) & (fincas_21.MODO == fincas[2])
                            &(fincas_21.TIPO==fincas[3])]
    finca_22 = fincas_22.loc[(fincas_22.ID_FINCA == fincas[0]) & (fincas_22.VARIEDAD == fincas[1]) & (fincas_22.MODO == fincas[2])
                            &(fincas_22.TIPO==fincas[3])]
    estimacion = (finca_20["PRODUCCION"].values/finca_20["SUPERFICIE"].values+finca_21["PRODUCCION"].values/finca_21["SUPERFICIE"].values)/2
    if((np.abs(finca_20["PRODUCCION"].values-finca_21["PRODUCCION"].values) <4000) | (np.abs(finca_20["PRODUCCION"].values/finca_20["SUPERFICIE"].values-finca_21["PRODUCCION"].values/finca_21["SUPERFICIE"].values)<2000)):
        train_bo.loc[finca_20.index,"ESTIMACION"] = estimacion
        train_bo.loc[finca_21.index,"ESTIMACION"] = estimacion
        train_bo.loc[finca_22.index,"ESTIMACION"] = estimacion
        fincas_20.loc[finca_20.index,"ESTIMACION"] = estimacion
        fincas_21.loc[finca_21.index,"ESTIMACION"] = estimacion
        fincas_22.loc[finca_22.index,"ESTIMACION"] = estimacion


In [30]:
fincas_22= fincas_22.loc[fincas_22.ESTIMACION == 0]
fincas_antes = train_bo[train_bo["CAMPAÑA"]<21]
keys = list(train_bo[["ID_FINCA","VARIEDAD","MODO","TIPO"]].columns.values)

i_22 = fincas_22.set_index(keys).index
i_21 = fincas_21.set_index(keys).index
i_antes = fincas_antes.set_index(keys).index

i_22 = i_22[~i_22.isin(i_antes)]
fincas_iguales_2 = i_22[i_22.isin(i_21)]
lista_indices = []
for fincas in fincas_iguales_2:
    finca_21 = fincas_21.loc[(fincas_21.ID_FINCA == fincas[0]) & (fincas_21.VARIEDAD == fincas[1]) & (fincas_21.MODO == fincas[2])
                            &(fincas_21.TIPO==fincas[3])]
    finca_22 = fincas_22.loc[(fincas_22.ID_FINCA == fincas[0]) & (fincas_22.VARIEDAD == fincas[1]) & (fincas_22.MODO == fincas[2])
                            &(fincas_22.TIPO==fincas[3])]

    estimacion = finca_21["PRODUCCION"].values/finca_21["SUPERFICIE"].values
    
    train_bo.loc[finca_21.index,"ESTIMACION"] = estimacion
    train_bo.loc[finca_22.index,"ESTIMACION"] = estimacion
    fincas_21.loc[finca_21.index,"ESTIMACION"] = estimacion
    fincas_22.loc[finca_22.index,"ESTIMACION"] = estimacion


#### Estimacion Campaña 22 con todas las fincas

In [31]:
fincas_22= fincas_22.loc[fincas_22.ESTIMACION == 0]
fincas_todas = train_bo[train_bo["CAMPAÑA"]<22]
keys = list(train_bo[["ID_FINCA","VARIEDAD","MODO","TIPO"]].columns.values)

i_22 = fincas_22.set_index(keys).index
i_todas = fincas_todas.set_index(keys).index

fincas_iguales_todas = i_22[i_22.isin(i_todas)]

for finca in fincas_iguales_todas:
    fincas = fincas_todas.loc[(fincas_todas.ID_FINCA == finca[0]) & (fincas_todas.VARIEDAD == finca[1]) & (fincas_todas.MODO == finca[2])
                            &(fincas_todas.TIPO==finca[3])]
    finca_22 = fincas_22.loc[(fincas_22.ID_FINCA == finca[0]) & (fincas_22.VARIEDAD == finca[1]) & (fincas_22.MODO == finca[2])
                            &(fincas_22.TIPO==finca[3])]

    estimacion = np.mean(fincas["PRODUCCION"]/fincas["SUPERFICIE"])
    
    train_bo.loc[fincas.index,"ESTIMACION"] = estimacion
    train_bo.loc[finca_22.index,"ESTIMACION"] = estimacion
    fincas_todas.loc[fincas.index,"ESTIMACION"] = estimacion
    fincas_22.loc[finca_22.index,"ESTIMACION"] = estimacion



In [32]:
fincas_22= fincas_22.loc[fincas_22.ESTIMACION == 0]
keys = list(train_bo[["ID_FINCA","VARIEDAD","TIPO"]].columns.values)

i_22 = fincas_22.set_index(keys).index
i_21 = fincas_21.set_index(keys).index
i_20 = fincas_20.set_index(keys).index

fincas_iguales_3 = i_22[i_22.isin(i_21)]
fincas_iguales_4 = i_22[i_22.isin(i_20)]

lista_indices = []
#Tendremos que mirar que el modo para todas sean diferentes, porque sino se tratará de un caso que antes hemos descartado de llenar
conjunto_20_21 = fincas_21.append(fincas_20)
medias_modo0 = np.mean(conjunto_20_21.loc[conjunto_20_21.MODO==0]["PRODUCCION"]/conjunto_20_21.loc[conjunto_20_21.MODO==0]["SUPERFICIE"])
medias_modo1 = np.mean(conjunto_20_21.loc[conjunto_20_21.MODO==1]["PRODUCCION"]/conjunto_20_21.loc[conjunto_20_21.MODO==1]["SUPERFICIE"])
# se espera una producción 50% mayor si es con modo 1 medias_modo1/medias_modo0

#Primero llenaremos aquellas que estan solo en la campaña 20 o 21, y luego las que estan en ambas (solo hay un caso en la campaña 21)
fincas_modo_21 = fincas_iguales_3[~fincas_iguales_3.isin(fincas_iguales_4)]
for fincas in fincas_modo_21:
    finca_21 = fincas_21.loc[(fincas_21.ID_FINCA == fincas[0]) & (fincas_21.VARIEDAD == fincas[1]) 
                            &(fincas_21.TIPO==fincas[2])]
    finca_22 = fincas_22.loc[(fincas_22.ID_FINCA == fincas[0]) & (fincas_22.VARIEDAD == fincas[1]) 
                            &(fincas_22.TIPO==fincas[2])]
    if (finca_21["MODO"].values[0] != finca_22["MODO"].values):
        estimacion = finca_21["PRODUCCION"].values[0]/finca_21["SUPERFICIE"].values[0]
        train_bo.loc[finca_21.index,"ESTIMACION"] = estimacion
        train_bo.loc[finca_22.index,"ESTIMACION"] = estimacion*finca_21["MODO"].values[0]/1.5 + estimacion*finca_22["MODO"].values*1.5 
        fincas_21.loc[finca_21.index,"ESTIMACION"] = estimacion
        fincas_22.loc[finca_22.index,"ESTIMACION"] = estimacion
    elif (len(finca_21["MODO"].values)>1):
        estimacion = finca_21["PRODUCCION"].values[1]/finca_21["SUPERFICIE"].values[1]
        train_bo.loc[finca_21.index,"ESTIMACION"] = estimacion
        train_bo.loc[finca_22.index,"ESTIMACION"] = estimacion/1.5
        fincas_21.loc[finca_21.index,"ESTIMACION"] = estimacion
        fincas_22.loc[finca_22.index,"ESTIMACION"] = estimacion
    
#Ahora aquellas que aparecen en las campañas 20,21 y 22    
fincas_iguales = i_22[i_22.isin(i_21[i_21.isin(i_20)])]
for fincas in fincas_iguales:
    finca_20 = fincas_20.loc[(fincas_20.ID_FINCA == fincas[0]) & (fincas_20.VARIEDAD == fincas[1]) 
                            &(fincas_20.TIPO==fincas[2])]
    finca_21 = fincas_21.loc[(fincas_21.ID_FINCA == fincas[0]) & (fincas_21.VARIEDAD == fincas[1]) 
                            &(fincas_21.TIPO==fincas[2])]
    finca_22 = fincas_22.loc[(fincas_22.ID_FINCA == fincas[0]) & (fincas_22.VARIEDAD == fincas[1]) 
                            &(fincas_22.TIPO==fincas[2])]
    
    finca_20 = finca_20.loc[finca_20.MODO.values != finca_22.MODO.values]
    finca_21 = finca_21.loc[finca_21.MODO.values != finca_22.MODO.values]

    modo_22 = finca_22["MODO"].values
    #comprovamos si hay algun modo diferente
    if (len(finca_20)==1 and len(finca_21)==1):
        estimacion = (finca_20["PRODUCCION"].values/finca_20["SUPERFICIE"].values+finca_21["PRODUCCION"].values/finca_21["SUPERFICIE"].values)/2
        train_bo.loc[finca_20.index,"ESTIMACION"] = estimacion
        train_bo.loc[finca_21.index,"ESTIMACION"] = estimacion
        train_bo.loc[finca_22.index,"ESTIMACION"] = estimacion/1.5 *(1-modo_22) + estimacion*1.5*modo_22
        fincas_20.loc[finca_20.index,"ESTIMACION"] = estimacion
        fincas_21.loc[finca_21.index,"ESTIMACION"] = estimacion
        fincas_22.loc[finca_22.index,"ESTIMACION"] = estimacion/1.5 *(1-modo_22) + estimacion*1.5*modo_22
    elif(len(finca_20)==1):
        estimacion = finca_20["PRODUCCION"].values/finca_20["SUPERFICIE"].values
        train_bo.loc[finca_20.index,"ESTIMACION"] = estimacion
        train_bo.loc[finca_22.index,"ESTIMACION"] = estimacion/1.5 *(1-modo_22) + estimacion*1.5*modo_22
        fincas_20.loc[finca_20.index,"ESTIMACION"] = estimacion
        fincas_22.loc[finca_22.index,"ESTIMACION"] = estimacion/1.5 *(1-modo_22) + estimacion*1.5*modo_22
    elif(len(finca_21)==1):
        estimacion = finca_21["PRODUCCION"].values/finca_21["SUPERFICIE"].values
        train_bo.loc[finca_21.index,"ESTIMACION"] = estimacion
        train_bo.loc[finca_22.index,"ESTIMACION"] = estimacion/1.5 *(1-modo_22) + estimacion*1.5*modo_22
        fincas_21.loc[finca_21.index,"ESTIMACION"] = estimacion
        fincas_22.loc[finca_22.index,"ESTIMACION"] = estimacion/1.5 *(1-modo_22) + estimacion*1.5*modo_22


#### Estimacion Campaña 22 con Variedad y Modo de Cultivo

In [33]:
fincas_22= fincas_22.loc[fincas_22.ESTIMACION == 0]
keys = list(train_bo[["VARIEDAD","MODO"]].columns.values)
i_22 = fincas_22.set_index(keys).index.drop_duplicates()
for comb in i_22:
    fincas = conjunto_20_21.loc[(conjunto_20_21.VARIEDAD == comb[0]) & (conjunto_20_21.MODO == comb[1])]
    finca_22 = fincas_22.loc[(fincas_22.VARIEDAD == comb[0]) & (fincas_22.MODO == comb[1])]

    estimacion = np.mean(fincas["PRODUCCION"]/fincas["SUPERFICIE"])
    train_bo.loc[finca_22.index,"ESTIMACION"] = estimacion
    fincas_22.loc[finca_22.index,"ESTIMACION"] = estimacion

#### Estimación todas las campañas

In [34]:
train_finca = train_bo[train_bo["CAMPAÑA"]<22]
keys = list(train_finca[["VARIEDAD","ID_FINCA","TIPO","MODO"]].columns.values)

fincas = train_finca.set_index(keys).index.drop_duplicates()

for com in fincas:
    estimacion = np.mean(train_finca.loc[(train_finca["VARIEDAD"]==com[0]) & (train_finca["ID_FINCA"]==com[1]) & (train_finca["TIPO"]==com[2]),"PRODUCCION"]/train_finca.loc[(train_finca["VARIEDAD"]==com[0]) & (train_finca["ID_FINCA"]==com[1]) & (train_finca["TIPO"]==com[2]),"SUPERFICIE"])
    if (len(train_finca.loc[(train_finca["VARIEDAD"]==com[0]) & (train_finca["ID_FINCA"]==com[1]) & (train_finca["TIPO"]==com[2])])>1):
        train_finca.loc[(train_finca["VARIEDAD"]==com[0]) & (train_finca["ID_FINCA"]==com[1]) & (train_finca["TIPO"]==com[2]) & (train_finca["ESTIMACION"]==0),"ESTIMACION"] = estimacion
        train_bo.loc[(train_bo["VARIEDAD"]==com[0]) & (train_bo["ID_FINCA"]==com[1]) & (train_bo["TIPO"]==com[2]) & (train_bo["ESTIMACION"]==0),"ESTIMACION"] = estimacion

In [35]:
fincas_21= train_finca.loc[(train_finca.ESTIMACION == 0 ) &  (train_finca.CAMPAÑA ==21)]
keys = list(train_bo[["VARIEDAD","MODO"]].columns.values)
i_21 = fincas_21.set_index(keys).index.drop_duplicates()
for comb in i_22:
    fincas = train_finca.loc[(train_finca.VARIEDAD == comb[0]) & (train_finca.MODO == comb[1])]
    finca_21 = fincas_21.loc[(fincas_21.VARIEDAD == comb[0]) & (fincas_21.MODO == comb[1])]
    estimacion = np.mean(fincas["PRODUCCION"]/fincas["SUPERFICIE"])
    train_bo.loc[finca_21.index,"ESTIMACION"] = estimacion
    fincas_21.loc[finca_21.index,"ESTIMACION"] = estimacion
train_bo = train_bo.loc[train_bo.ESTIMACION !=0]

#### Factor Producción

In [36]:
train_bo["ESTIMACION_PRODUCCION"] = train_bo["ESTIMACION"]*train_bo["SUPERFICIE"]
train_bo["FACTOR_PRODUCCION"] = train_bo["PRODUCCION"]/train_bo["ESTIMACION_PRODUCCION"]
cambio_signo = -1/train_bo[train_bo["FACTOR_PRODUCCION"]<1]["FACTOR_PRODUCCION"]
train_bo.loc[train_bo.FACTOR_PRODUCCION<1,"FACTOR_PRODUCCION"] = cambio_signo+1
train_bo.loc[train_bo.FACTOR_PRODUCCION>0,"FACTOR_PRODUCCION"] = train_bo.loc[train_bo.FACTOR_PRODUCCION>0,"FACTOR_PRODUCCION"]-1

In [37]:
index_outliers = train_bo.index[np.abs(train_bo["FACTOR_PRODUCCION"])>5]
train_bo = train_bo[~train_bo.index.isin(index_outliers)]

#### Quitamos campañas 14 y 15

In [38]:
train_bo= train_bo.loc[(train_bo.CAMPAÑA != 14) & (train_bo.CAMPAÑA !=15)]


## ETO Y METEO DATASET

### ETO

#### Formato Fecha y eliminación de datos innecesarios

In [39]:
ETO_dataset['date'] = pd.to_datetime(ETO_dataset['date'], format="%Y%m%d")
for i,j in enumerate(ETO_dataset.keys()):
    if j == "date" :
        continue
    elif j == 'ID_ESTACION':
        continue
    elif "Day" not in j:
        ETO_dataset.drop(j, axis=1, inplace=True)
    elif "Daytime" in j:
        ETO_dataset.drop(j, axis=1, inplace=True)

In [40]:
ETO_dataset=ETO_dataset.drop('FeelsLikeLocalDayAvg', axis=1)
ETO_dataset=ETO_dataset.drop('FeelsLikeLocalDayMax', axis=1)
ETO_dataset=ETO_dataset.drop('FeelsLikeLocalDayMin', axis=1)

In [41]:
ETO_dataset['Week'] = ETO_dataset['date'].dt.week
ETO_dataset['Month'] = ETO_dataset['date'].dt.month
ETO_dataset['Year'] = ETO_dataset['date'].dt.year

### Imputation de los valores NaN con modelos

In [42]:
ETO_dataset = ETO_dataset.drop(["EvapotranspirationLocalDayMax","EvapotranspirationLocalDayMin","EvapotranspirationLocalDayMin","EvapotranspirationLocalDayMax",
                "GlobalHorizontalIrradianceLocalDayMax","GlobalHorizontalIrradianceLocalDayMin","MSLPLocalDayMax","MSLPLocalDayMin","GustLocalDayMax","GustLocalDayMin"],axis=1)

#### Evatranspiración

In [43]:
Evatransporation = ETO_dataset.drop(["MSLPLocalDayAvg","GustLocalDayAvg","GlobalHorizontalIrradianceLocalDayAvg","date","ID_ESTACION"],axis=1)
X = Evatransporation[~Evatransporation.isnull().any(axis=1)]
target_y = X["EvapotranspirationLocalDayAvg"]
Pred_X = Evatransporation[Evatransporation.isnull().any(axis=1)].drop("EvapotranspirationLocalDayAvg",axis=1)

In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score,confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel


X_train, X_test, y_train, y_test = train_test_split(X.drop(["EvapotranspirationLocalDayAvg"],axis=1), (target_y*10).astype(int)
, test_size=0.3, random_state=99)
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X_train, y_train)
model = SelectFromModel(lsvc, prefit=True)
X_train = model.transform(X_train)
X_test = model.transform(X_test)
regr = RandomForestClassifier(n_estimators = 1000,max_depth=25, random_state=0,n_jobs=-1)
regr.fit(X_train,y_train)

prediccio = regr.predict(X_test)

X_pred = model.transform(Pred_X)
EV_pred =regr.predict(X_pred)
Pred_X["EvapotranspirationLocalDayAvg"] = EV_pred/10
ETO_dataset.loc[ETO_dataset.index.isin(Pred_X["EvapotranspirationLocalDayAvg"].index),"EvapotranspirationLocalDayAvg"] = Pred_X["EvapotranspirationLocalDayAvg"]



#### Presion

In [45]:
Presion = ETO_dataset.drop(["GustLocalDayAvg","GlobalHorizontalIrradianceLocalDayAvg","date","ID_ESTACION","Week","Year"],axis=1)
X = Presion[~Presion.isnull().any(axis=1)]
target_y = X["MSLPLocalDayAvg"]
Pred_X = Presion[Presion.isnull().any(axis=1)].drop("MSLPLocalDayAvg",axis=1)

In [46]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.linear_model import Lasso

X_train, X_test, y_train, y_test = train_test_split(X.drop("MSLPLocalDayAvg",axis=1), target_y
, test_size=0.3, random_state=99,shuffle=True)
lasso = Lasso(alpha=0.3,fit_intercept=False).fit(X_train,y_train)
model = SelectFromModel(lasso, prefit=True)
X_train = model.transform(X_train)
X_test = model.transform(X_test)
regr = RandomForestRegressor(n_estimators = 200,max_depth=25, random_state=0,n_jobs=-1)
regr.fit(X_train,y_train)

prediccio = regr.predict(X_test)
X_pred = model.transform(Pred_X)
Pres_pred =regr.predict(X_pred)
Pred_X["MSLPLocalDayAvg"] = Pres_pred
ETO_dataset.loc[ETO_dataset.index.isin(Pred_X["MSLPLocalDayAvg"].index),"MSLPLocalDayAvg"] = Pred_X["MSLPLocalDayAvg"]


#### Radiacion

In [47]:
Radiacion = ETO_dataset.drop(["GustLocalDayAvg","date","ID_ESTACION","Year"],axis=1)
X = Radiacion[~Radiacion.isnull().any(axis=1)]
target_y = X["GlobalHorizontalIrradianceLocalDayAvg"]
Pred_X = Radiacion[Radiacion.isnull().any(axis=1)].drop("GlobalHorizontalIrradianceLocalDayAvg",axis=1)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X.drop("GlobalHorizontalIrradianceLocalDayAvg",axis=1), target_y
                                                    , test_size=0.3, random_state=99,shuffle=True)
lasso = Lasso(alpha=0.3,fit_intercept=False).fit(X_train,y_train)
model = SelectFromModel(lasso, prefit=True)
X_train = model.transform(X_train)
X_test = model.transform(X_test)
regr = RandomForestRegressor(n_estimators = 200,max_depth=30, random_state=0,n_jobs=-1)
regr.fit(X_train,y_train)

prediccio = regr.predict(X_test)

In [49]:
X_pred = model.transform(Pred_X)
Rad_pred =regr.predict(X_pred)
Pred_X["GlobalHorizontalIrradianceLocalDayAvg"] = Rad_pred
ETO_dataset.loc[ETO_dataset.index.isin(Pred_X["GlobalHorizontalIrradianceLocalDayAvg"].index),"GlobalHorizontalIrradianceLocalDayAvg"] = Pred_X["GlobalHorizontalIrradianceLocalDayAvg"]


Quitamos la lluvia mínima diaria y la nievie mínima diaria

In [50]:
ETO_dataset.drop(["PrecipAmountLocalDayMin","SnowAmountLocalDayMin"], axis=1, inplace=True)
#Ni mínimo de lluvia diaria ni de nieve aporta nada, ya que sera 0 en la mayoría de casos


### METEO

#### Formato Fecha y eliminación de datos innecesarios

In [51]:
def separate_by_space(string: str, index: int):
    word_list = string.split() #converts string into a list of strings
    return word_list[index]

METEO_dataset['validTimeUtc'] = METEO_dataset['validTimeUtc'].astype('str')

date = pd.DataFrame(METEO_dataset['validTimeUtc'].apply(lambda x: separate_by_space(x, 0)))
METEO_dataset.insert(loc=0, column='date', value=date['validTimeUtc'])

hour = pd.DataFrame(METEO_dataset['validTimeUtc'].apply(lambda x: separate_by_space(x, 1)))
METEO_dataset.insert(loc=1, column='Hour', value=hour['validTimeUtc'])

#METEO_dataset.drop('validTimeHour', axis=1, inplace=True)

METEO_dataset['date'] = METEO_dataset['date'].str.replace('-', '')
METEO_dataset['date'] = pd.to_datetime(METEO_dataset['date'], format="%Y%m%d")
METEO_dataset.drop('validTimeUtc', axis=1, inplace=True) # Ya no nos hace falta porque tenemos Date y Hour
ETO_dataset['date'] = pd.to_datetime(ETO_dataset['date'], format="%Y%m%d")

In [52]:
METEO_dataset['precip1Hour'].fillna(method='ffill',inplace=True)
METEO_dataset['snow1Hour'].fillna(method='ffill',inplace=True)

In [53]:
# Volume of rain
suma_prec = METEO_dataset.groupby(['date','ID_ESTACION'])['precip1Hour'].agg(np.sum).reset_index().rename(columns={'precip1Hour':'precipDay'})
ETO_dataset = ETO_dataset.merge(suma_prec,how="left", on=["date","ID_ESTACION"])

In [54]:
ETO_dias = ETO_dataset.copy()


### Series Temporales Para Llenar los meses de julio a octubre del 2022

In [55]:
ETO_series = ETO_dias[["date","ID_ESTACION","TemperatureLocalDayAvg",
                           "GlobalHorizontalIrradianceLocalDayAvg","WindSpeedLocalDayMax","precipDay"]]

In [56]:
Series_Temporales = []

for id_estacion in np.unique(ETO_series["ID_ESTACION"]):
    Series_Temporales.append(ETO_series[ETO_series["ID_ESTACION"]==id_estacion])

In [57]:
for i,serie in enumerate(Series_Temporales):
    serie = serie.append(pd.DataFrame({'date': pd.date_range(start="2022-06-30", periods=130, freq='D', closed='right')}))
    Series_Temporales[i] = serie.reset_index().drop(['index','ID_ESTACION'],axis=1)

In [58]:
from prophet import Prophet
for i,serie in enumerate(Series_Temporales):
    for col in serie.columns:
        if col == "date":
            pass
        else:
            data = serie[serie['date']<"2022-07-01"][['date',col]].rename(columns={'date':'ds',col:'y'})
            m=Prophet()    
            m.fit(data)
            future = m.make_future_dataframe(periods=129)
            forecast = m.predict(future)[['ds', 'yhat']]
            forecast = forecast[forecast["ds"]>"2022-06-30"]
            Series_Temporales[i].loc[forecast.index,col] = forecast['yhat']


11:22:31 - cmdstanpy - INFO - Chain [1] start processing
11:22:32 - cmdstanpy - INFO - Chain [1] done processing
11:22:33 - cmdstanpy - INFO - Chain [1] start processing
11:22:33 - cmdstanpy - INFO - Chain [1] done processing
11:22:34 - cmdstanpy - INFO - Chain [1] start processing
11:22:34 - cmdstanpy - INFO - Chain [1] done processing
11:22:34 - cmdstanpy - INFO - Chain [1] start processing
11:22:34 - cmdstanpy - INFO - Chain [1] done processing
11:22:35 - cmdstanpy - INFO - Chain [1] start processing
11:22:36 - cmdstanpy - INFO - Chain [1] done processing
11:22:37 - cmdstanpy - INFO - Chain [1] start processing
11:22:37 - cmdstanpy - INFO - Chain [1] done processing
11:22:37 - cmdstanpy - INFO - Chain [1] start processing
11:22:38 - cmdstanpy - INFO - Chain [1] done processing
11:22:38 - cmdstanpy - INFO - Chain [1] start processing
11:22:38 - cmdstanpy - INFO - Chain [1] done processing
11:22:39 - cmdstanpy - INFO - Chain [1] start processing
11:22:39 - cmdstanpy - INFO - Chain [1]

11:23:38 - cmdstanpy - INFO - Chain [1] done processing
11:23:39 - cmdstanpy - INFO - Chain [1] start processing
11:23:39 - cmdstanpy - INFO - Chain [1] done processing
11:23:40 - cmdstanpy - INFO - Chain [1] start processing
11:23:40 - cmdstanpy - INFO - Chain [1] done processing
11:23:40 - cmdstanpy - INFO - Chain [1] start processing
11:23:40 - cmdstanpy - INFO - Chain [1] done processing
11:23:41 - cmdstanpy - INFO - Chain [1] start processing
11:23:42 - cmdstanpy - INFO - Chain [1] done processing
11:23:42 - cmdstanpy - INFO - Chain [1] start processing
11:23:43 - cmdstanpy - INFO - Chain [1] done processing
11:23:43 - cmdstanpy - INFO - Chain [1] start processing
11:23:43 - cmdstanpy - INFO - Chain [1] done processing
11:23:44 - cmdstanpy - INFO - Chain [1] start processing
11:23:44 - cmdstanpy - INFO - Chain [1] done processing


In [59]:
Series_Temporales[0]['ID_ESTACION'] = 0
ETO_series_temporales = Series_Temporales[0]
for i,serie in enumerate(Series_Temporales[1:]):
    serie['ID_ESTACION']= i+1
    ETO_series_temporales= ETO_series_temporales.append(serie)

## Predicción final con datos meteorologicos

In [60]:
ETO_series_temporales["Year"] = ETO_series_temporales['date'].dt.year
ETO_series_temporales['Month'] = ETO_series_temporales['date'].dt.month
ETO_series_temporales['Week'] = ETO_series_temporales['date'].dt.week

Dataset Lluvia

In [61]:
ETO_Lluvia = ETO_series_temporales[["precipDay","Year","Month","ID_ESTACION"]]
mask1 = ETO_Lluvia["Month"]>10
ETO_Lluvia.loc[mask1, 'Year'] = ETO_Lluvia.loc[mask1]['Year']+1
ETO_Lluvia = ETO_Lluvia.groupby(["ID_ESTACION","Year","Month"]).agg(np.sum).reset_index()
ETO_Lluvia = ETO_Lluvia[(ETO_Lluvia["Year"]>2015) & (ETO_Lluvia["Year"]<2023)]
ETO_Lluvia = ETO_Lluvia.pivot_table("precipDay", ['Year', 'ID_ESTACION'], 'Month')
ETO_Lluvia.columns = ["Month_"+str(col) for col in ETO_Lluvia.columns.values]
ETO_Lluvia.reset_index(inplace=True)
ETO_Lluvia["Year"]=ETO_Lluvia["Year"]-2000
ETO_Lluvia.rename(columns={"Year":"CAMPAÑA"},inplace=True)

In [62]:
ETO_Lluvia["Precipitacion_Brotacion"] = ETO_Lluvia["Month_3"] +  ETO_Lluvia["Month_4"] + ETO_Lluvia["Month_5"]
ETO_Lluvia["Precipitacion_Crecimiento"] = ETO_Lluvia["Month_6"] +  ETO_Lluvia["Month_7"] + ETO_Lluvia["Month_8"]
ETO_Lluvia["Precipitacion_Maduracion"] = ETO_Lluvia["Month_9"] +  ETO_Lluvia["Month_10"]
ETO_Lluvia["Precipitacion_Reserva"] = ETO_Lluvia["Month_11"] +  ETO_Lluvia["Month_12"] + ETO_Lluvia["Month_1"] +  ETO_Lluvia["Month_2"]

In [63]:
ETO_Lluvia=ETO_Lluvia[["CAMPAÑA","ID_ESTACION","Precipitacion_Reserva","Precipitacion_Maduracion","Precipitacion_Crecimiento","Precipitacion_Brotacion"]]

Dataset Temperatura, Radiación y Viento

In [64]:
ETO_series_temporales = ETO_series_temporales[(ETO_series_temporales["Month"]>2) & (ETO_series_temporales["Month"]<11)]
ETO_Clima = ETO_series_temporales.groupby(["ID_ESTACION","Year","Month"]).agg(np.mean).reset_index()

In [65]:
ETO_Clima = ETO_Clima[(ETO_Clima["Year"]>2015) & (ETO_Clima["Year"]<2023)]


def modify_data(feature):
    data = ETO_Clima.pivot_table(feature, ['Year', 'ID_ESTACION'], 'Month')
    data.columns = ["Month_"+str(col) for col in data.columns.values]
    data.reset_index(inplace=True)
    data["Year"]=data["Year"]-2000

    data.rename(columns={"Year":"CAMPAÑA"},inplace=True)
    return data

ETO_Temperatura = modify_data("TemperatureLocalDayAvg")
ETO_Viento = modify_data("WindSpeedLocalDayMax")
ETO_Radiacion = modify_data("GlobalHorizontalIrradianceLocalDayAvg")

In [66]:
ETO_Temperatura["Temperatura_Brotacion"] = (ETO_Temperatura["Month_3"] +  ETO_Temperatura["Month_4"] + ETO_Temperatura["Month_5"])/3
ETO_Temperatura["Temperatura_Crecimiento"] = (ETO_Temperatura["Month_6"] +  ETO_Temperatura["Month_7"] + ETO_Temperatura["Month_8"] )/3
ETO_Temperatura["Temperatura_Maduracion"] = (ETO_Temperatura["Month_9"] +  ETO_Temperatura["Month_10"])/2

ETO_Temperatura=ETO_Temperatura[["CAMPAÑA","ID_ESTACION","Temperatura_Brotacion","Temperatura_Crecimiento","Temperatura_Maduracion"]]

ETO_Viento["Viento_Brotacion"] = (ETO_Viento["Month_3"] +  ETO_Viento["Month_4"] + ETO_Viento["Month_5"])/3
ETO_Viento["Viento_Crecimiento"] = ( ETO_Viento["Month_6"] +  ETO_Viento["Month_7"] + ETO_Viento["Month_8"])/3
ETO_Viento["Viento_Maduracion"] = ( ETO_Viento["Month_9"] +  ETO_Viento["Month_10"])/2

ETO_Viento=ETO_Viento[["CAMPAÑA","ID_ESTACION","Viento_Brotacion","Viento_Crecimiento","Viento_Maduracion"]]

ETO_Radiacion["Radiacion_Brotacion"] = (ETO_Radiacion["Month_3"] +  ETO_Radiacion["Month_4"] + ETO_Radiacion["Month_5"])/3
ETO_Radiacion["Radiacion_Crecimiento"] = (ETO_Radiacion["Month_6"] +  ETO_Radiacion["Month_7"] + ETO_Radiacion["Month_8"])/3
ETO_Radiacion["Radiacion_Maduracion"] = (ETO_Radiacion["Month_9"] +  ETO_Radiacion["Month_10"])/2

ETO_Radiacion=ETO_Radiacion[["CAMPAÑA","ID_ESTACION","Radiacion_Brotacion","Radiacion_Crecimiento","Radiacion_Maduracion"]]


### Modelos Finales/ Ensemble de modelos

#### Random Forest

In [67]:
train_dataset = train_bo[(train_bo["CAMPAÑA"]>15)]
train_dataset = train_dataset.merge(ETO_Lluvia,how="left",on=["CAMPAÑA","ID_ESTACION"])
train_dataset= train_dataset.merge(ETO_Viento,how="left",on=["CAMPAÑA","ID_ESTACION"])
train_dataset= train_dataset.merge(ETO_Radiacion,how="left",on=["CAMPAÑA","ID_ESTACION"])

train_set_data = train_dataset[(train_dataset["CAMPAÑA"]<22)]
pred_set_data = train_dataset[train_dataset["CAMPAÑA"]==22]

train_set_data.drop(["CAMPAÑA","ID_FINCA","ID_ESTACION","ID_ZONA"],axis=1,inplace=True)

train_set_data = train_set_data[np.abs(train_set_data["FACTOR_PRODUCCION"])<1.6]

pred_set = pred_set_data.drop(["PRODUCCION","FACTOR_PRODUCCION","CAMPAÑA","ID_FINCA","ID_ESTACION","ID_ZONA"],axis=1)

X_train = train_set_data.drop(["PRODUCCION","FACTOR_PRODUCCION"],axis=1)
y_train = train_set_data["PRODUCCION"]


regr = RandomForestRegressor(n_estimators = 300,max_depth=25, min_samples_split =6, random_state=0, n_jobs=-1)
regr.fit(X_train,y_train)
prediccion_rf = regr.predict(pred_set)

#### XGBoost

In [68]:
from xgboost import XGBRegressor

train_dataset = train_bo[(train_bo["CAMPAÑA"]>15)]
train_dataset = train_dataset.merge(ETO_Lluvia,how="left",on=["CAMPAÑA","ID_ESTACION"])
train_dataset= train_dataset.merge(ETO_Temperatura,how="left",on=["CAMPAÑA","ID_ESTACION"])
train_dataset= train_dataset.merge(ETO_Viento,how="left",on=["CAMPAÑA","ID_ESTACION"])
train_dataset= train_dataset.merge(ETO_Radiacion,how="left",on=["CAMPAÑA","ID_ESTACION"])

train_set_data = train_dataset[(train_dataset["CAMPAÑA"]<22)]
pred_set_data = train_dataset[train_dataset["CAMPAÑA"]==22]

train_set_data.drop(["CAMPAÑA","ID_FINCA","ID_ESTACION","ID_ZONA"],axis=1,inplace=True)

train_set_data = train_set_data[np.abs(train_set_data["FACTOR_PRODUCCION"])<1.6]

pred_set = pred_set_data.drop(["PRODUCCION","FACTOR_PRODUCCION","CAMPAÑA","ID_FINCA","ID_ESTACION","ID_ZONA"],axis=1)

X_train = train_set_data.drop(["PRODUCCION","FACTOR_PRODUCCION"],axis=1)
y_train = train_set_data["PRODUCCION"]


regr = XGBRegressor(n_estimators = 90,reg_lambda=0.8,min_child_weight=5,subsample = 0.7,colsample_bytree=0.7,booster='gbtree',eta=0.1,max_depth=3, tree_method="exact",objective = "reg:squarederror",
                         eval_metric = "rmse").fit(X_train,y_train)
    
prediccion_xgb = regr.predict(pred_set)


In [69]:
prediccion_estimacion = pred_set["ESTIMACION_PRODUCCION"]

In [70]:
prediccion_final = 0.18 * prediccion_estimacion + 0.14 * prediccion_xgb + 0.68 * prediccion_rf

### Entrega

In [71]:
pred_set_data["PRODUCCION"] = prediccion_final

In [72]:
entrega = pred_set_data[["ID_FINCA","VARIEDAD","MODO","TIPO","COLOR","SUPERFICIE","PRODUCCION"]]

In [73]:
entrega = entrega.sort_values(["ID_FINCA","VARIEDAD","MODO","TIPO","COLOR","SUPERFICIE","PRODUCCION"])

In [74]:
entrega["MODO"] = entrega["MODO"]+1
entrega["PRODUCCION"]=entrega["PRODUCCION"].round(2)
entrega["SUPERFICIE"]=entrega["SUPERFICIE"].round(2)

entrega.to_csv("SOFIS-ticados.txt",index=False,header=False,sep = "|")